## IMPORTS

In [1]:
import torch
import torchvision

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer

import os

## DATAROOT

In [2]:
dataroot = './data/planet/'
train_root = os.path.join(dataroot,'train-jpg/')
test_root = os.path.join(dataroot,'test-jpg/')
train_label_root = os.path.join(dataroot,'train_classes.csv')
test_label_root = os.path.join(dataroot,'test_classes.csv')

## HYPERPARAMS

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 256
num_workers = 4
pin_memory = True
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(32),    
    torchvision.transforms.ToTensor(),
])
lr = 0.01
momentum = 0.5

In [4]:
df = pd.read_csv(train_label_root)
len(df["image_name"])

40479

In [5]:
img = Image.open(train_root+df["image_name"][0]+".jpg")
# img = img.convert('RGB')

## DATASET

In [6]:
class PlanetDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, lbl_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        
        temp_df = pd.read_csv(lbl_dir)
        self.mlb = MultiLabelBinarizer()
        self.X_train = temp_df['image_name']
        self.y_train = self.mlb.fit_transform(temp_df['tags'].str.split()).astype(np.float32)
    def __len__(self):
        return len(self.X_train)
        
    def __getitem__(self, index):
        img = Image.open(train_root+df["image_name"][index]+".jpg")
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        label = torch.from_numpy(self.y_train[index])
        return img, label

In [7]:
train_ds = PlanetDataset(train_root, train_label_root, transform)

In [8]:
len(train_ds)

40479

## DATALOADER

In [9]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

In [10]:
for Xb, yb in train_dl:
    print(Xb.shape, yb.shape)
    break

torch.Size([256, 3, 32, 32]) torch.Size([256, 17])


## MODEL

In [11]:
import torch.nn as nn
import torch.nn.functional as F

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

model = Net().to(device)

## OPTIMIZER

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr, momentum)

## LOSS

In [14]:
criterion = torch.nn.BCELoss()

## TRAIN

In [15]:
def train(epoch):
    model.train()
    for batch_idx, (Xb, yb) in enumerate(train_dl):
        Xb = Xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        out = model(Xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(Xb), len(train_dl.dataset),
                100. * batch_idx / len(train_dl), loss.data.item()))

In [16]:
for epoch in range(1, 2):
    train(epoch)

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/40479 (0%)]	Loss: 0.698192
Train Epoch: 1 [2560/40479 (6%)]	Loss: 0.691304
Train Epoch: 1 [5120/40479 (13%)]	Loss: 0.682352
Train Epoch: 1 [7680/40479 (19%)]	Loss: 0.672952
Train Epoch: 1 [10240/40479 (25%)]	Loss: 0.662355
Train Epoch: 1 [12800/40479 (31%)]	Loss: 0.648452
Train Epoch: 1 [15360/40479 (38%)]	Loss: 0.630642
Train Epoch: 1 [17920/40479 (44%)]	Loss: 0.602465
Train Epoch: 1 [20480/40479 (50%)]	Loss: 0.561632
Train Epoch: 1 [23040/40479 (57%)]	Loss: 0.517396
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.469789
Train Epoch: 1 [28160/40479 (69%)]	Loss: 0.422600
Train Epoch: 1 [30720/40479 (75%)]	Loss: 0.379018
Train Epoch: 1 [33280/40479 (82%)]	Loss: 0.360094
Train Epoch: 1 [35840/40479 (88%)]	Loss: 0.357801
Train Epoch: 1 [38400/40479 (94%)]	Loss: 0.342202
